# This code is the song detector module of the song detector pipeline, and this one applies an amplitude threshold.
 ### This ONLY uses an amplitude threshold to figure out when there is song inside of a .wav file, then saves a .json file with the .wav file name and the time segments containing detected song as an output.

 ### For now, it also saves a .npz file with the amplitude_trace, smoothed_amplitude, and 

In [ ]:
import numpy as np
import os
import json
from pathlib import Path
from scipy.io import wavfile
from scipy.signal import spectrogram, windows, ellip, filtfilt
from scipy.ndimage import gaussian_filter1d
from itertools import groupby
from operator import itemgetter


def process_wav_file(file_path, output_npz_folder, low_cut=500, high_cut=8000,
                     sigma=100, threshold=0.05, segment_duration=10):

    base_name = Path(file_path).stem
    samplerate, data = wavfile.read(file_path)
    if data.ndim > 1:
        data = data.mean(axis=1)

    # Bandpass filter
    nyquist = samplerate / 2
    wp = [low_cut / nyquist, high_cut / nyquist]
    b, a = ellip(5, 0.2, 40, wp, btype='band')
    data = filtfilt(b, a, data)

    duration_seconds = data.shape[0] / samplerate
    segment_length_samples = int(segment_duration * samplerate)
    num_segments = int(np.ceil(duration_seconds / segment_duration))

    all_amplitude = []
    all_smoothed = []
    detected_segments = []

    for i in range(num_segments):
        start_sample = i * segment_length_samples
        end_sample = start_sample + segment_length_samples
        segment_data = np.zeros(segment_length_samples, dtype=data.dtype)
        if start_sample < data.shape[0]:
            segment_data[:max(0, min(segment_length_samples, data.shape[0] - start_sample))] = data[start_sample:end_sample]

        f, t, Sxx = spectrogram(
            segment_data,
            fs=samplerate,
            window=windows.gaussian(2048, std=2048/8),
            nperseg=2048,
            noverlap=(2048 - 119)
        )

        freq_mask = (f >= low_cut) & (f <= high_cut)
        Sxx_band = Sxx[freq_mask, :]
        amplitude_trace = np.sum(Sxx_band, axis=0)
        smoothed_trace = gaussian_filter1d(amplitude_trace, sigma=sigma)
        is_song = smoothed_trace > threshold

        if np.any(is_song):
            song_times = t + (i * segment_duration)
            song_indices = np.where(is_song)[0]
            for k, g in groupby(enumerate(song_indices), lambda x: x[0] - x[1]):
                group = list(map(itemgetter(1), g))
                start_time = song_times[group[0]]
                end_time = song_times[group[-1]]
                detected_segments.append([float(start_time), float(end_time)])

        all_amplitude.extend(amplitude_trace)
        all_smoothed.extend(smoothed_trace)

    # Save .npz file WITHOUT spectrogram
    npz_out_path = os.path.join(output_npz_folder, f"{base_name}_traces.npz")
    np.savez(npz_out_path,
             amplitude_trace=np.array(all_amplitude),
             smoothed_trace=np.array(all_smoothed),
             bandpass_low_cut=low_cut,
             bandpass_high_cut=high_cut,
             Gaussian_smoothing_sigma=sigma,
             amplitude_threshold=threshold)

    return {
        "file_name": Path(file_path).name,
        "file_path": str(file_path),
        "detected_segments": detected_segments
    }


def batch_process_folder(folder_path, segment_duration=10, sigma=100, threshold=0.05,
                         low_cut=500, high_cut=8000):
    
    folder_path = Path(folder_path)
    parent_dir = folder_path.parent
    folder_name = folder_path.name
    output_folder = parent_dir / f"{folder_name}_amplitude_filter_song_detection_outputs"
    
    os.makedirs(output_folder, exist_ok=True)
    npz_folder = output_folder / "npz_traces"
    os.makedirs(npz_folder, exist_ok=True)

    wav_files = list(folder_path.glob("*.wav"))
    if not wav_files:
        print("No .wav files found.")
        return

    print(f"🔍 Found {len(wav_files)} .wav files in {folder_name}")

    metadata = []
    for wav_file in wav_files:
        info = process_wav_file(
            wav_file,
            output_npz_folder=npz_folder,
            low_cut=low_cut,
            high_cut=high_cut,
            sigma=sigma,
            threshold=threshold,
            segment_duration=segment_duration
        )
        metadata.append(info)

    json_path = output_folder / "amplitude_filter_detected_song_segments.json"
    with open(json_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"✅ Done. Metadata saved to: {json_path}")
    print(f"✅ Traces saved to: {npz_folder}/")

In [3]:
# === Example Usage ===
folder_path = '/Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5510_unsegmented_songs/55'
batch_process_folder(
    folder_path,
    segment_duration=10,
    sigma=50,
    threshold=800,
    low_cut=500,
    high_cut=8000
)


🔍 Found 359 .wav files in 55
✅ Done. Metadata saved to: /Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5510_unsegmented_songs/55_song_detection_outputs/detected_song_segments.json
✅ Traces saved to: /Users/mirandahulsey-vincent/Documents/allPythonCode/BYOD_class/data_inputs/USA5510_unsegmented_songs/55_song_detection_outputs/npz_traces/
